In [57]:
import pandas as pd
import numpy as np

In [58]:
store_inventory = pd.read_csv('Supplemental Order Store Data.csv')
forecast = pd.read_csv('Supplemental Order Forecast Data.csv')
available_inventory = pd.read_csv('Available Inventory.csv')

## Calculating Total Pipeline

In [59]:
store_inventory['store_in_transit_quantity_this_year'] = pd.to_numeric(store_inventory['store_in_transit_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_in_warehouse_quantity_this_year'] = pd.to_numeric(store_inventory['store_in_warehouse_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_on_hand_quantity_this_year'] = pd.to_numeric(store_inventory['store_on_hand_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_on_order_quantity_this_year'] = pd.to_numeric(store_inventory['store_on_order_quantity_this_year'], errors='coerce').fillna(0)

# Now create the 'total_pipeline' column
store_inventory['total_pipeline'] = (
    store_inventory['store_in_transit_quantity_this_year'] + 
    store_inventory['store_in_warehouse_quantity_this_year'] + 
    store_inventory['store_on_hand_quantity_this_year'] + 
    store_inventory['store_on_order_quantity_this_year']
)

## Calculating Needed Quantity

In [61]:
# Initialize an empty list to store the data
data = []

# getting unique list of items
items = store_inventory['vendor_stock_id'].unique()

for item in items:
    all_links_item_number = store_inventory[store_inventory['vendor_stock_id'] == item]['all_links_item_number'].values[0]

    # filtering the list of stores once we select the item
    inventory_filtered_item = store_inventory[store_inventory['vendor_stock_id'] == item]

    # sorting the stores based on the total pipeline ascending
    inventory_filtered_item = inventory_filtered_item.sort_values(by='total_pipeline', ascending=True)

    # getting the unique list of stores
    stores = inventory_filtered_item['store_number'].unique()

    total_shipped = 0
    for store in stores:
        # getting the total pipeline and vendor pack quantity for the selected variables
        inventory_filtered = inventory_filtered_item[inventory_filtered_item['store_number'] == store]
        total_pipeline = inventory_filtered['total_pipeline'].values[0]
        vendor_pack_quantity = inventory_filtered['vendor_pack_quantity'].values[0]
        max_shelf_quantity = inventory_filtered['max_shelf_quantity_this_year'].values[0]
        store_on_hand_quantity = inventory_filtered['store_on_hand_quantity_this_year'].values[0]

        # getting the forecasted quantity for the selected variables
        forecast_filtered = forecast[(forecast['vendor_stock_id'] == item) & (forecast['store_nbr'] == store)]
        if not forecast_filtered.empty:
            forecasted_quantity = forecast_filtered['final_fcst_each_qty'].values[0]
        else:
            forecasted_quantity = 0  # Handle the case when no forecast is available

        # calculating the needed quantity for the item and store
        needed_quantity = total_pipeline - forecasted_quantity
        #max_shelf_minus_pipeline = max_shelf_quantity - total_pipeline
        if needed_quantity > 0:
            needed_quantity = 0
        else:
            needed_quantity = abs(needed_quantity)
            whse_packs_needed = np.ceil(needed_quantity / vendor_pack_quantity)
            needed_quantity = whse_packs_needed * vendor_pack_quantity
            #needed_quantity = needed_quantity - max_shelf_quantity
            if needed_quantity > max_shelf_quantity:
                needed_quantity = np.ceil(max_shelf_quantity / vendor_pack_quantity)
            else:
                #needed_quantity = needed_quantity
                needed_quantity = np.ceil(needed_quantity / vendor_pack_quantity)

        # Append the result to the list, including all_links_item_number
        data.append({
            'Item': item,
            'Store': store,
            'Needed Quantity': needed_quantity,
            'All Links Item Number': all_links_item_number
        })
# Convert the list of dictionaries into a DataFrame
needed_quantity_df = pd.DataFrame(data)

# remove zeros
needed_quantity_df = needed_quantity_df[needed_quantity_df['Needed Quantity'] > 0]

## Sorting by Item and then Needed Quantity

This ensures that we send to the stores that need it most first.

In [62]:
# sort the dataframe based on item then needed quantity
needed_quantity_df = needed_quantity_df.sort_values(by=['All Links Item Number', 'Needed Quantity'], ascending=[True, False])

# SUM THE NEEDED QUANTITY FOR EACH ITEM
total_quantity_needed = needed_quantity_df.groupby(['All Links Item Number']).sum()
total_quantity_needed

,Item,Store,Needed Quantity
All Links Item Number,,,
585320691,646503,280809,122.0
597319350,343525,157238,65.0
659879048,1352448,1381140,903.0
663269028,1605786,1565045,836.0
663269050,1467075,1494468,952.0
663398046,3261013,3366958,2060.0


## Calculating the Fullfilled Amount

In [64]:
# Function to handle the row-by-row subtraction
def subtract_needed_quantity_by_item(needed_quantity_df, available_inventory):
    # Loop through each unique item in the needed_quantity_df
    for item in needed_quantity_df['Item'].unique():
        # Get the total available value for the item from available_inventory
        available_quantity = available_inventory.loc[available_inventory['Item'] == str(item), 'Available - split '].values
        print(f"{item}:{available_quantity}")
        # If the item is in available_inventory
        if available_quantity.size > 0:
            available_quantity = available_quantity[0]  # Get the available quantity for the item
        else:
            available_quantity = 0  # If the item is not found, set available quantity to 0
        # Loop through the rows of needed_quantity_df for this item
        for idx, row in needed_quantity_df[needed_quantity_df['Item'] == item].iterrows():
            needed_qty = row['Needed Quantity']
            
            # Check if we have enough in the total available quantity to subtract the needed quantity
            if available_quantity >= needed_qty:
                # If enough, subtract the needed quantity from available_quantity
                available_quantity -= needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity'] = needed_qty  # Mark fully fulfilled
            else:
                # If not enough, fulfill what is available and set available_quantity to 0
                needed_quantity_df.at[idx, 'Fulfilled Quantity'] = available_quantity  # Fulfill partially
                available_quantity = 0  # Set available_quantity to 0 after using it up
            
            # If available_quantity is exhausted, break the loop
            if available_quantity == 0:
                break
    
    return needed_quantity_df

# Apply the function to subtract from available inventory based on needed quantity
fulfilled_df = subtract_needed_quantity_by_item(needed_quantity_df, available_inventory)

# Add Remaining Quantity column
fulfilled_df['Remaining Quantity'] = fulfilled_df['Needed Quantity'] - fulfilled_df['Fulfilled Quantity']

5343:[0.]
5285:[1292.]
2304:[-1.]
2249:[-1.]
2325:[17.]
2341:[0.]


## Finalizing the Dataset

In [65]:
# drop the rows with greater than 0 remaining quantity
fulfilled_df = fulfilled_df[fulfilled_df['Fulfilled Quantity'] > 0]

## Overview of the Need and Fulfilled Quantity

In [67]:
total_fulfilled_quantity = fulfilled_df.groupby(['Item']).sum()
total_fulfilled_quantity = total_fulfilled_quantity.drop(columns=['Store','All Links Item Number','Remaining Quantity'])
total_fulfilled_quantity

,Needed Quantity,Fulfilled Quantity
Item,,
2325,19.0,17.0
5285,65.0,65.0


## Exporting the Data as a CSV

In [68]:
# drop the columns that are not needed
fulfilled_df = fulfilled_df.drop(columns=['Needed Quantity','Remaining Quantity'])
fulfilled_df

fulfilled_df.to_csv('suppy.csv', index=False)